In [18]:
import sqlite3
import requests
import time
from random import choice
import threading
import numpy as np

def clean_pool(pool):
    for ip in pool:
        proxy = {'http':ip}
        test_url = r'http://www.qq.com/'
        try:
            res = requests.get(test_url, proxies = proxy, timeout = 10)
        except:
            del pool[pool.index(ip)]

        try:
            res.raise_for_status()
        except:
            if ip in pool:
                del pool[pool.index(ip)]
    return pool

def get_data(num):
    start = 50*(num-1) + 1
    end = 50*num + 1
    front = r'http://club.excelhome.net/forum.php?mod=forumdisplay&fid=2&orderby=dateline&orderby=dateline&filter=author&page='
    for i in range(start, end):
        url = front + str(i)
        time.sleep(1)
        proxy = {'http':np.random.choice(real_ip_pool, size = 1)[0]}
        try:
            res = requests.get(url, proxies = proxy, timeout = 10)
            res.raise_for_status()
            res.encoding = 'gbk'
            filename = r'C:\Users\asus\Desktop\iPython\Projects\Untitled Folder\data\page{}'.format(str(i)) + '.txt'
            with open(filename, 'wb') as f:
                for it in res.iter_content(100000):
                    f.write(it)
        except:
            pass
        
ip_get_api = r'http://vtp.daxiangdaili.com/ip/?tid=558461663228484&num=50&operator=2&filter=on'
database_path = r'c:\users\asus\desktop\ipython\ip_pool.db'
                               
conn = sqlite3.connect(database_path)
select_query = '''
    SELECT
        *
    FROM
        IP_ADDRESS;'''

cursor = conn.cursor()
ip_pool = [i[1] for i in cursor.execute(select_query).fetchall()]

real_ip_pool = clean_pool(ip_pool)
                               
while len(real_ip_pool) < 30:
    res = requests.get(ip_get_api)
    res.raise_for_status()
    for line in res.text.split():
        real_ip_pool.append(line)
    real_ip_pool = clean_pool(real_ip_pool)

cursor.execute('''DELETE FROM IP_ADDRESS;''')
for ip in real_ip_pool:
    query = '''INSERT INTO IP_ADDRESS VALUES(NULL, ''' + ip.join('\'\'') + ');'
    cursor.execute(query)
conn.commit()
cursor.close()
conn.close()

threads = []
for i in range(1, 6):
    threadobj = threading.Thread(target = get_data, args = [i])
    threadobj.start()
    threads.append(threadobj)

for i in threads:
    i.join()

print('all threads are runing, good luck')

all threads are runing, good luck
